In [1]:
'''
<<<design>>>

1. input 형태 조정 *****
 1.-(1) MFCC 파일을 x, y로 구분하여 리스트에 저장
 (x : 컬럼 차원 리스트, y : 클래스 라벨(1차원 리스트))
2. input을 받아서 svm kernel function을 바꿔서 돌리기
 2-(2) 클래스 여러 개인 부분 조정
 2-(3) kernel function 변경
 
 pickle 을 통해 학습값 저장 후 활용

'''

from sklearn import svm
from sklearn.datasets import make_blobs
from random import *

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import librosa
import librosa.display
from matplotlib.pyplot import specgram
import os
import IPython.display as ipd
import random
import pickle


In [2]:

#.wav 파일을 받아와서 MFCC로 변환
# mean 을 통하여 정규화를 해서 집어넣는다고 함.
def wav2mfcc(path):
    mfcc_x = []
    label_y = []
    for subdir, dirs, files in os.walk(path):
        for file in files:
            try:
                X, sample_rate = librosa.load(os.path.join(subdir,file), res_type = "kaiser_fast", duration = 2.5, sr = 22050*2, offset = 0.5)
                mfcc = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc = 40).T,axis=0)
                y = int(file[7:8])-1
                mfcc_x.append(mfcc)
                label_y.append(y)
            except ValueError:
                continue

    return mfcc_x, label_y


#list를 판다스 df로 보여주기
def print_df(mylist):
    df = pd.DataFrame(data = mylist)
    print(df)


def train_test_split(x, y, train_size):
    
    # 두 리스트를 합쳐서, 셔플하여 조정
    # 이후 train size 만큼 나눠서 분배
    
    tmp = [[a,b] for a, b in zip(x, y)]
    random.shuffle(tmp)
    x2 = [n[0] for n in tmp]
    y2 = [n[1] for n in tmp]
    index = int(len(y) * train_size)
    x_train = []
    x_test = []
    x_train = x2[:index]
    x_test = x2[index:]
    y_train = y2[:index]
    y_test = y2[index:]
    '''
        x_train = x[:index]
    x_test = x[index:]
    y_train = y[:index]
    y_test = y[index:]

    '''


    return x_train, x_test, y_train, y_test

def accuracy_score(error,total):
    return error / total


def extract_mfcc(wav_file_name):
    '''This function extracts mfcc features and obtain the mean of each dimension
    Input : path_to_wav_file
    Output: mfcc_features'''
    y, sr = librosa.load(wav_file_name)
#     trimmed_data = np.zeros((160, 20))
    mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T,axis=0)
#     data = np.array(librosa.feature.mfcc(y = y, sr = sr, n_mfcc=40).T)
#     if data.shape[0] <= 160:
#         trimmed_data[:data.shape[0],0:] = data[:,0:]
#     else:
#         trimmed_data[0:,0:] = data[0:160,0:]
    return mfccs


In [3]:
##### load data from savee dataset
root_dir = "./savee-database/AudioData/"
# root_dir = "../input/audio_speech_actors_01-24/"
savee_data = []
savee_labels = []
for actor_dir in sorted(os.listdir(root_dir)):
    if actor_dir[-4:] == ".txt":
        continue
    for file_name in os.listdir(os.path.join(root_dir, actor_dir)):
        if file_name[0] == "c":
            continue
        wav_file_name = os.path.join(root_dir, actor_dir, file_name)
        savee_data.append(extract_mfcc(wav_file_name))
        #01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).
        if file_name[0] == "n":
            savee_labels.append(0)
        if file_name[0] == "a":
            savee_labels.append(4)
        if file_name[0] == "d":
            savee_labels.append(6)
        if file_name[0] == "f":
            savee_labels.append(5)
        if file_name[0] == "h":
            savee_labels.append(2)
        if file_name[:2] == "sa":
            savee_labels.append(3)
        if file_name[:2] == "su":
            savee_labels.append(7)

In [4]:
#### convert data to array and make labels categorical
savee_data_array = np.asarray(savee_data)
savee_label_array = np.array(savee_labels)
savee_data_array.shape
# savee_data_array.shape

(480, 40)

In [5]:
##### load data from emo dataset

root_dir = "./emo-db/wav"
# root_dir = "../input/audio_speech_actors_01-24/"
emo_data = []
emo_labels = []
for file_name in os.listdir(root_dir):
    wav_file_name = os.path.join(root_dir, file_name)
    emo_data.append(extract_mfcc(wav_file_name))
    #01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).
    # anger
    if file_name[5] == "W":
        emo_labels.append(4)
    #boredom
    if file_name[5] == "L":
        emo_labels.append(1)
    #disgust
    if file_name[5] == "E":
        emo_labels.append(6)
    #anxiety (fear 와 비슷하다고 판단하여 추가)
    if file_name[5] == "A":
        emo_labels.append(5)
    #happy
    if file_name[5] == "F":
        emo_labels.append(2)
    #sad
    if file_name[5] == "T":
        emo_labels.append(3)
    #neutral
    if file_name[5] == "N":
        emo_labels.append(0)
    


In [6]:
emo_data_array = np.asarray(emo_data)
emo_label_array = np.array(emo_labels)
emo_data_array.shape

(535, 40)

In [7]:


'''
-------------------------------------------------
                      main
-------------------------------------------------
'''

#input
'''
listData = readCSVtoList('output.csv') #전역변수랑 로컬변수랑 이름 같아도 되나?
x, x_name = get_x(listData)
y = get_y(listData)
'''

# 모든 데이터 집어넣음
x, y = wav2mfcc("C:\\Users\\JW\\JPNotebook\\O2DD\\ravdess-video\\")
x2, y2 = wav2mfcc("C:\\Users\\JW\\JPNotebook\\O2DD\\ravdess-emotional-speech-audio\\")
x3, y3 = wav2mfcc("C:\\Users\\JW\\JPNotebook\\O2DD\\ravdess-emotional-song-audio\\")







In [8]:
x_data = np.array(x)
y_labels= np.array(y)
x2_data = np.array(x2)
y2_labels= np.array(y2)
x3_data = np.array(x3)
y3_labels= np.array(y3)

data = np.r_[x_data, x2_data, x3_data]
labels = np.r_[y_labels, y2_labels, y3_labels]

data.shape


#data split
x_train, x_test, y_train, y_test = train_test_split(data, labels, 0.70)

#print(x_train)
print_df(x_train)
print_df(y_train)
print_df(x_test)
print_df(y_test)


              0           1          2          3          4          5   \
0    -619.222670   64.177939 -38.507848  10.326028 -16.548701  -7.445555   
1    -493.605883   78.403124 -59.328538   2.917562  -5.959734 -15.284565   
2    -419.720852   73.028260 -56.681549   4.046819  -6.625583  -8.569353   
3    -465.032207   70.907345 -23.506550   5.622329  -7.250303   0.564889   
4    -553.802471   70.648811 -26.657246   6.857753  -7.232124   2.864682   
...          ...         ...        ...        ...        ...        ...   
3427 -591.851919   82.116318   7.755493  17.411675   1.099998  10.513944   
3428 -450.481321   80.544637 -24.184132  -2.557460 -20.147437  12.804345   
3429 -648.262485  113.865457  -5.345855  18.631916   2.353619  15.293511   
3430 -434.584039   78.319575 -15.550904  16.307647  -9.961129  11.075298   
3431 -515.527545  119.673687 -26.632437  13.125755   2.965245  19.398560   

             6          7          8          9   ...        30        31  \
0    -28.6

In [9]:

#training session
clf = svm.SVC(kernel='linear')
clf.fit(x_train, y_train)


In [19]:
'''
with open('data.pickle', 'wb') as f:
    pickle.dump(clf, f)
'''

In [29]:
'''
clf2 = svm.SVC(kernel='linear')

with open('data.pickle', 'rb') as f:
    clf2 = pickle.loads(f)
'''

"\nclf2 = svm.SVC(kernel='linear')\n\nwith open('data.pickle', 'rb') as f:\n    clf2 = pickle.loads(f)\n"

In [30]:

#test session
y_pred = clf.predict(x_test)
print('총 테스트 개수: %d, 오류개수:%d' %(len(y_test), (y_test != y_pred).sum()))

print("Score : ", clf.score(x_test,y_test))
#print('정확도: %.2f' % accuracy_score(y_test,y_pred))

# 비고
'''
트레이닝 데이터와 테스트 데이터의 비율은 생각보다는 크게 영향이 없었다.

'''

총 테스트 개수: 1472, 오류개수:562
Score :  0.6182065217391305


'\n트레이닝 데이터와 테스트 데이터의 비율은 생각보다는 크게 영향이 없었다.\n\n'

In [31]:

savee_pred = clf.predict(savee_data_array)
print('총 테스트 개수: %d, 오류개수:%d' %(len(savee_label_array), (savee_label_array != savee_pred).sum()))

print("Score : ", clf.score(savee_data_array,savee_label_array))

총 테스트 개수: 480, 오류개수:368
Score :  0.23333333333333334


In [32]:

emo_pred = clf.predict(emo_data_array)
print('총 테스트 개수: %d, 오류개수:%d' %(len(emo_label_array), (emo_label_array != emo_pred).sum()))

print("Score : ", clf.score(emo_data_array,emo_label_array))

총 테스트 개수: 535, 오류개수:445
Score :  0.16822429906542055
